In [1]:
# %%
import pandas as pd
import os

# Ordnerpfad, in dem sich die CSV-Dateien befinden
ordner_pfad = "C:/Users/anasl/Desktop/Daten 1/Daten/Output Data"

# Alle CSV-Dateien im Ordner auflisten
csv_dateien = [datei for datei in os.listdir(ordner_pfad) if datei.endswith('.csv')]
alle_csv_daten = []

# Alle CSV-Dateien einlesen (mit low_memory=False zur Vermeidung von DtypeWarnings)
for datei_name in csv_dateien:
    vollstaendiger_pfad = os.path.join(ordner_pfad, datei_name)
    daten = pd.read_csv(vollstaendiger_pfad, sep=",", encoding='latin1', low_memory=False)
    alle_csv_daten.append(daten)

# Alle eingelesenen Daten kombinieren
kombinierte_csv_daten = pd.concat(alle_csv_daten, ignore_index=True)


In [2]:
# %%
# Datum konvertieren
kombinierte_csv_daten['Datum'] = pd.to_datetime(kombinierte_csv_daten['Datum'], errors='coerce')

# Nach Standort und Datum sortieren
sortierte_daten = kombinierte_csv_daten.sort_values(['Standort', 'Datum']).reset_index(drop=True)


In [3]:
# %%
# Funktion zur automatischen Korrektur von codierten Zeichen (z. B. Ã¼ → ü)
def fix_encoding(text):
    if pd.isna(text):
        return text
    try:
        return text.encode('latin1').decode('utf-8')
    except:
        return text  # Falls Umwandlung fehlschlägt, Original zurückgeben

# Auf die Spalte „Standort“ anwenden
sortierte_daten['Standort'] = sortierte_daten['Standort'].apply(fix_encoding)


In [4]:
# %%
# Bekannte fehlerhafte Ortsnamen manuell ersetzen
spezial_fixes = {
    'GroÃŸ': 'Groß',
    'MÃ¼nster': 'Münster',
    'OsnabrÃ¼ck': 'Osnabrück',
    'SchÃ¶nau am KÃ¶nigssee Bayern': 'Schönau am Königssee Bayern',
    'Ulm-MÃ¤hringen': 'Ulm-Mähringen'
}

def manuelle_korrektur(text):
    for falsch, richtig in spezial_fixes.items():
        text = text.replace(falsch, richtig)
    return text

# Auf die Spalte „Standort“ anwenden
sortierte_daten['Standort'] = sortierte_daten['Standort'].apply(manuelle_korrektur)


In [5]:
# %%
# Exportieren als UTF-8-SIG für Excel-Kompatibilität
sortierte_daten.to_csv(
    "C:/Users/anasl/Desktop/Daten 1/Daten/Output Data/Weather_Master_Data.csv",
    index=False,
    encoding='utf-8-sig'
)
